# Segmentation Train code - SHSH version

### init by KK

<p>샘플 이미지 예 (왼 : ori, 오른 lbl)</p>
<div style="display: inline-block;">
    <img src='./dataset/ori/Abyssinian_1.jpg' width = 300 height = 300 alt='ori_img_sample' style="display: inline-block;">
    <img src='./dataset/lbl/Abyssinian_1.png' width = 300 height = 300 alt='ori_img_sample' style="display: inline-block;">
</div>

<p>sample data : <a href=https://www.robots.ox.ac.uk/~vgg/data/pets/>https://www.robots.ox.ac.uk/~vgg/data/pets/</a></p>
<p>위 데이터의 lbl이미지는 원래는 1채널, onehot 이미지이나(검은 이미지) 임의의 색으로 바꾸어 데이터 셋을 새로 만들었습니다.

# 0. import

## 0.1 basic import

<p><u>또한 같은 기능으로 사용되는데 서로 다른 함수를 호출하는 경우도 있었습니다... 속도의 차이인지 작성한 사람이 달라서 그런지 모르겠지만 일단 함수는 그대로 두었습니다.</u></p>또한 변수명을 짓는 스타일이 서로 달라서 혼동되는 변수명도 있습니다. 주의하세요

In [1]:
import os

import sys
import gc

import time
import datetime
import random
import math

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2 as cv
from PIL import Image
import numpy as np

## 0.2 tf import

In [2]:
import tensorflow as tf
from tensorflow import keras

import tensorflow.keras.backend as K
import tensorflow_addons as tfa # 0.11.2?

print(tf.config.list_physical_devices('GPU')) # GPU 유뮤 확인

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-09-25 17:42:56.548169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 17:42:56.607735: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-25 17:42:56.608031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## 0.3 sm import

In [3]:
%env SM_FRAMEWORK=tf.keras

import segmentation_models as sm # pip install segmentation_models
sm.set_framework('tf.keras')
print(sm.framework())

from segmentation_models.losses import * 
from segmentation_models.base import Loss
from segmentation_models.base import functional as F

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.
tf.keras


## 0.4 tf.keras.layers import

In [4]:
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
# check call

# 1. Global Params

학습에 사용될 전역변수들을 설정합니다. (모델 관련 전역변수는 아래에 있습니다.)  

In [5]:
RGB_LABELMAP = {
    # Background
    0 : {'RGB' : (50,50,50), 'name': 'bg', 'target': 0},

    # animal
    1 : {'RGB' : (120,50,75), 'name': 'animal', 'target': 1},
    2 : {'RGB' : (128,255,128), 'name': 'border', 'target': 2},
#     3 : {'RGB' : (50,50,80), 'name': 'border', 'target': 1} # Merge의 예
}

# RGB_LABELMAP : 학습하기 전 Onehot이미지를 만들거나, 학습 중 혹은 테스트 결과로 반환되는 Onehot이미지를
#               다시 RGB이미지로 만들 때 사용됩니다. target은 합쳐야 할(merge) 클래스를 설정할 때도 사용됩니다.

TARGET_CLASSES  = [0,1,2]
# TARGET_CLASSES  = [0,2]

# TARGET_CLASSES : 학습에 해당되는 클래스리스트 입니다. 다만 RGB_LABELMAP의 내부 dict의 key인 'target'과는 다른 의미입니다.
# 1번 클래스를 2번 클래스로 Merge를 했더라도 2번위치도 학습해야하므로 [0,1,2]가 되어야합니다.

CLASS_CNT = len(TARGET_CLASSES) 
# 이전에는 수동으로 int를 입력했었으나 이제는 리스트의 길이를 사용하는 걸로 바꾸었습니다.

# COLOR_MODE = ['RGB']
SRC_CHANNELS = 3 # 예전에 NSXY이미지가 있었던 시절 '4'일때도 있어서 따로 정의한 변수입니다. 사실 아래에 '3'으로 입력해도 됩니다.

# 2. Tools

## 2.1 Image Tools

### 2.1.1 RGB2Onehot

In [6]:
'''
   RGB2Onehot(): 일반 RGB이미지를  OneHot이미지로 바꿉니다. 만약 학습되는 클래스가 3개라면 (h,w,3), 10개라면 (h,w,10) numpy.ndarray를 반환합니다.
'''

def RGB2OneHot(input_image:np.ndarray, label_map:dict, bChannel:bool = False) -> np.ndarray :
    # global CLASS_CNT
    reshape_image = input_image.reshape((-1,3))

    if bChannel :
        dense = np.zeros(reshape_image.shape[:1]+(CLASS_CNT,), np.uint8)
    else : 
        dense = np.zeros(reshape_image.shape[:1]+(1,), np.uint8)
    
#     print(f'\t{dense.shape}')

    for label, sub_dict in label_map.items():
        target_label = sub_dict['target']

        if (target_label not in TARGET_CLASSES) :
            print(f'label is not in ... -> BG')
            target_label = 0
        
        color = list(sub_dict['RGB'])
        label_name = sub_dict['name']

#         print(reshape_image[0])
#         print(color)

        if bChannel :
            temp_np = np.zeros(CLASS_CNT)
            temp_np[target_label] = 1
            dense[np.all(reshape_image == color, axis=-1)] = temp_np
        else :
            dense[np.all(reshape_image == color, axis=-1)] = target_label
            
    if bChannel :
        dense = dense.reshape((input_image.shape[:2] + (CLASS_CNT,)))
    else :
        dense = dense.reshape((input_image.shape[:2] + (1,)))
            
    return dense

### 2.1.2 onehot2RGB

In [7]:
'''
   onehot2RGB(): OneHot이미지를 RGB이미지로 바꿉니다. (제 기억엔) 학습에는 쓰이지 않고 결과를 볼때만 사용됩니다.
'''

def onehot2RGB(input_image:np.ndarray, label_map:dict, bChannel:bool) ->np.ndarray :
    rgb_lbl = np.zeros(input_image.shape+(3,), np.uint8)

    if bChannel == True :
        rgb_lbl = np.zeros(input_image.shape[:2] + (3,), np.uint8)
        input_image = input_image.argmax(-1)

    for label, sub_dict in label_map.items() :
        color = sub_dict['RGB']
        label_name = sub_dict['name']

        rgb_lbl[input_image == label] = color

    rgb_lbl = rgb_lbl.reshape(input_image.shape[:2]+ (3,))

    return rgb_lbl

### 2.1.3 show_color

In [8]:
'''
    학습에 쓰이는 Color를 단순히 출력하는 함수
'''

def show_color() :
    cell = np.zeros((3,3,3), np.uint8)
    idx = 0

    f, axes = plt.subplots(1, CLASS_CNT, figsize=(18,4))

    for label, sub_dict in RGB_LABELMAP.items():
        if label in TARGET_CLASSES :
            color = sub_dict['RGB']
            label_name = sub_dict['name']
            target_num = sub_dict['target']
            cell[:] = color

            axes[idx].imshow(cell)
            axes[idx].set_xlabel(f'{label_name} : {label}')
            idx=idx+1
        else :
            pass

    plt.show()

In [ ]:
show_color()

### 2.1.4 show_mask()

In [ ]:
'''
    show_mask() : 마스크 이미지를 만들때(create_mask()) 사용되는 함수인데, 이름이 왜 show인지는...
'''
def show_mask(pred_val:np.ndarray, num=0):
    print('pred val.shape = ', pred_val.shape)
    b, w, h, ch = pred_val.shape 

    f, axes = plt.subplots(1, ch, figsize =(18, 6))
    for idx in range(ch) :
        axes[idx].imshow(pred_val[0][:,:,idx] * 255, cmap='gray')

### 2.1.4 create_mask()

In [ ]:
'''
    create_mask() : 마스크 이미지를 만듭니다.
'''

def create_mask(pred_val:np.ndarray, num=0):
    
    show_mask(pred_val)

    pred_mask = tf.argmax(pred_val, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[num]

### 2.1.5 calculate_class_iou()

In [ ]:
'''
    각 클래스별 iou를 구하여 반환합니다.
'''

def calculate_class_iou(gt_labels, pred_labels, class_list = None, preview = False) :
    
    img_size = pred_labels.shape[:2]

    if class_list is None or len(class_list) == 0 :
        class_list = np.arange(CLASS_CNT)

    gt_selected_labels = np.zeros(img_size, dtype='uint8')
    pred_selected_labels = np.zeros(img_size, dtype='uint8')

    for class_num in class_list :
        np.place(gt_selected_labels, gt_labels == class_num, 255)
        np.place(pred_selected_labels, pred_labels == class_num, 255)

    iou_list = []

    for class_num in class_list :
        gt_cur_labels = np.zeros(img_size, dtype=np.uint8)
        pred_cur_labels = np.zeros(img_size, dtype=np.uint8)

        np.place(gt_cur_labels, gt_labels == class_num, 1)
        np.place(pred_cur_labels, pred_labels == class_num, 1)

        intersection = gt_cur_labels * pred_cur_labels
        union = gt_cur_labels + pred_cur_labels - intersection

        gt_sum = gt_cur_labels.sum()
        intersection_sum = intersection.sum()
        union_sum = union.sum()

        if union_sum == 0 :
            iou = 0
        else :
            iou = intersection_sum / union_sum
        iou = np.round(iou, 2)

        print(f'class [{class_num}]={iou}\t({intersection_sum}/{gt_sum}/{union_sum})')
        iou_list.append(iou)

    return iou_list, class_list

        # cnt_gt, labels_gt, stats_gt, centroids_gt = cv.connectedComponentsWithStats(gt_selected_labels)
        # cnt_pred, labels_pred, stats_pred, centroids_pred = cv.connectedComponentsWithStats(pred_selected_labels)

        # gt_num_blobs = len(np.unique())

# 3. Dataset

## 3.1 set dir

In [ ]:
base_path = './dataset/'

ORI_EXT = '.jpg' # 오리지날 이미지 확장명
LBL_EXT = '.png' # 레이블 이미지 확장명

ori_path = base_path + 'ori/'
lbl_path = base_path + 'lbl/'

ori_list = [item for item in os.listdir(ori_path) if ORI_EXT in item]
lbl_list = [item for item in os.listdir(lbl_path) if LBL_EXT in item]

print('all image\'s cnt :', len(ori_list))
print('all label\'s cnt :', len(lbl_list))

In [ ]:
INPUT_SHAPE = (320,320,3)
print(INPUT_SHAPE)

In [ ]:
val_list_txt_path = base_path + 'val_list.txt'
print(val_list_txt_path)
print('\tval_list_txt is exist :', os.path.isfile(val_list_txt_path))

## 3.2 valid data utils

### 3.2.1 load_valid_label_list

In [ ]:
'''
   load_valid_label_list() : 미리 작성한 텍스트파일에서 검증파일 리스트를 불러옵니다. 
'''

def load_valid_label_list(val_txt_path:str) -> list:
    val_label_list = []

    f = open(val_txt_path)
    while(True) :
        line = f.readline().replace('\n', '')
        '''
            custom commentary
            '#'으로 시작하는 line은 무시하고
            '@'으로 시작하는 line은 '@'뒤를 무시
            ./dataset/val_list.txt 에  예시가 있어요.
        ''' 
        if '#' in line :
            continue
        if '@'in line :
            line = line.split('@')[0]
        if not line :
            break
        else :
            val_label_list.append(line)
    f.close()

    return val_label_list 


### 3.2.2. load_train_file_list()

In [ ]:
'''
   load_train_file_list() : 전체셋 - 검증셋 = 학습셋
'''

def load_train_label_list(all_label_file_list:list, valid_label_list:list) -> list:
    except_list = valid_label_list

    load_train_label_list = [item for item in all_label_file_list if item not in except_list]

    return load_train_label_list

# 3.3. make Dataset

<p>아래 함수들에 문제가 살짝 있습니다...</p>
<p>(조금 이야기가 길어요... 사실 이거 때문에 주석을 단다고 한건데...)</p>
<br>
<p>데이터의 크기가 많아지면서 OOM이 뜨다보니</p>
<br>
<p>"야, 데이터제너레이터 좀 만들어봐라"</p>
<br>
<p>라고 하셔서 DataGenerator를 만들었습니다.</p>
<p>학습이 시작되면 각 Batch마다 "__len__()"번 만큼 "__getitem__()"을 호출하여 데이터를 가져옵니다.(numpy.ndarray로 변환합니다)</p>
<p>그렇게 한 번에 불러오는 데이터의 양을 적게 해서 OOM을 피하는 건데</p>
<br>
<p>"야, 굳이 이미지를 numpy.ndarray로 변환 해놓은 데이터를 날리고 또 변환하는 건 너무 오래 걸리고 답답하다. 그러니까..."</p>
<br>
<p>라고 하셔서 모든 이미지를 각각 numpy.ndarray 로 변환 후 하나의 <u>리스트</u>에 담아두고</p>
<p>인덱스로 간단히 가져와서 이미지를 numpy.ndarray로 바꾸는 작업을 처음 한 번만 진행해도 되어서 학습속도를 높일 수 있었습니다.</p>
<br>
<p>하지만 생각해보니 수많은 numpy.ndarray을 담을 수 있는 <u>리스트</u>는 컴퓨터에 일반 RAM에 저장되는 거 같아요.</p>
<p>OOM이라고 뜨진 않지만 컴퓨터가 죽으려고 했거든요. (16GB RAM)</p>
<p>생각해보니 우리가 지금까지 본 OOM은 그래픽카드의 VRAM이 터진다는 의미였어요.</p>
<p>단지 우리가 사용해 오던 워크스테이션 PC의 RAM의 용량이 엄청 커서 우리가 느끼지 못했던 거였네요.</p>
<br>
<p>일단은 폴더안에 파일을 줄이는 걸로 테스트만 가능하게끔 해놨습니다.
<br>
<p>그러면 다시 DataGenerator를 되돌여야하는데 (학습 속도는 다시 느려지겠지만) 그건 아직 못했습니다.

### 3.3.1 mk_ori_list()

In [ ]:
'''
    mk_ori_list() : original 이미지를 numpy.ndarray로 바꾸어 리스트에 담아 반환합니다.
                    위에서 언급했듯 RAM에 무리가 갈거 같아서 데이터의 수를 줄였습니다.
'''

def mk_ori_list(file_name_list:list, output_type:str = 'list') :
    if output_type == 'list' : 
        ori_img_list = []

        for idx, file_name in enumerate(file_name_list) :
            print(f'\rProcessing... \"{file_name}\" \t', end='')
            out = []
            if True : 
                raw_ori_img_path = os.path.join(ori_path, file_name.replace(LBL_EXT, ORI_EXT))
                raw_ori_src = cv.imread(raw_ori_img_path)

                raw_ori_src = cv.cvtColor(raw_ori_src, cv.COLOR_BGR2RGB)
                
                raw_ori_src = raw_ori_src.astype(np.float32)
                raw_ori_src = cv.resize(raw_ori_src, INPUT_SHAPE[:2], interpolation=cv.INTER_NEAREST)
                raw_ori_src = raw_ori_src / 255.0

                out.append(raw_ori_src)
            
            final_ori_src = np.concatenate(out, -1)
            ori_img_list.append(final_ori_src)

        print('')
    
    return ori_img_list

### 3.3.2 mk_lbl_list

In [ ]:
'''
    mk_lbl_list() : original 이미지를 numpy.ndarray로 바꾸어 리스트에 담아 반환합니다.
                    위에서 언급했듯 RAM에 무리가 갈거 같아서 데이터의 수를 줄였습니다.
'''

def mk_lbl_list(file_name_list:list, output_type:str = 'list') :
    if output_type == 'list' : 
        lbl_img_list = []

        for idx, file_name in enumerate(file_name_list) :
            print(f'\rProcessing... {file_name} \t', end='')
            
            raw_lbl_img_path = os.path.join(lbl_path, file_name)
            raw_lbl_src = cv.imread(raw_lbl_img_path)

            raw_lbl_src = cv.cvtColor(raw_lbl_src, cv.COLOR_BGR2RGB)

            final_lbl_src  = RGB2OneHot(raw_lbl_src, RGB_LABELMAP, bChannel=True)
            final_lbl_src = cv.resize(final_lbl_src, INPUT_SHAPE[:2], interpolation=cv.INTER_NEAREST)
            final_lbl_src = final_lbl_src.astype(np.float32)
            
            lbl_img_list.append(final_lbl_src)

        print('')
    
    return lbl_img_list


### CustomDG

In [ ]:
'''
    CustomDataGenerator : 만들어 놓긴 했지만 이미 만들어진 리스트에 담겨있는 numpy.ndarray을 인덱싱해서 가져오는 역할만 할 뿐입니다.
                        일반 RAM이 터지지 않게 다시 바꾸려면 
                        self.train_x_set 에는 파일경로 리스트를 (현재 : numpy.ndarray가 담겨있는 리스트)
                        self.train_y_set 도 위와 동일
                        
                        __getitem__() 은 cv.imread()등으로 이미지파일을 numpy.ndarray로 바꾸는 과정이 필요합니다. 
                            (mk_ori_list() 안의 함수들이 들어갈거에요. mk_ori_list()만들어질 때 __getitem__()안의 내용을 가져왔었으니.)
'''

class CustomDataGenerator(Sequence) :
    def __init__(self, train_x_set:list, train_y_set:list, batch_size:int=16, channel_num:int=3, image_size:tuple=(0,0), shuffle:bool=False, rotation_range:float=0, brighteness_range:float=0, v_flip:bool=False, h_flip:bool=False) :
        self.train_x_set = train_x_set
        self.train_y_set = train_y_set
        self.batch_size = batch_size
        self.channel_num = channel_num
        self.shuffle = shuffle
        self.image_size = image_size
        self.rotation_range = rotation_range
        self.brighteness_range = brighteness_range
        self.v_flip = v_flip
        self.h_flip = h_flip

        self.indices = np.arange(len(train_x_set))

    def __len__(self) :
        return math.ceil(len(self.train_x_set) / self.batch_size)

    def __getitem__(self, idx:int):
        start_idx = idx*self.batch_size
        end_idx = (idx+1)*self.batch_size

        curr_indices = self.indices[start_idx:end_idx]

        batch_ori_list = [self.train_x_set[idx] for idx in curr_indices]
        batch_lbl_list = [self.train_y_set[idx] for idx in curr_indices]

        result_ori_list = []
        result_lbl_list = []

        for idx, _img in enumerate(batch_ori_list) :

            ori_img = batch_ori_list[idx]
            lbl_img = batch_lbl_list[idx]

            tf_ori_img = tf.convert_to_tensor(ori_img)
            tf_lbl_img = tf.convert_to_tensor(lbl_img)

            # aug pass

            result_ori_list.append(tf.expand_dims(tf_ori_img, 0))
            result_lbl_list.append(tf.expand_dims(tf_lbl_img, 0))

        result_ori_list = tf.concat(result_ori_list, 0)
        result_lbl_list = tf.concat(result_lbl_list, 0)
        
        return result_ori_list, result_lbl_list

    def on_epoch_end(self):
        random.shuffle(self.indices)


# 4. make Dataset

In [ ]:
val_list = load_valid_label_list(val_list_txt_path)
len(val_list)

In [ ]:
train_list = load_train_label_list(lbl_list, val_list)
len(train_list)

In [ ]:
'''
    여기서 일반 RAM이 터집니당...
'''

mk_train_ori_list_start = time.time()
train_ori_list = mk_ori_list(train_list, output_type='list')
print('mk_train_ori_list took', time.time() - mk_train_ori_list_start)

mk_train_lbl_list_start = time.time()
train_lbl_list = mk_lbl_list(train_list, output_type='list')
print('mk_train_lbl_list took', time.time() - mk_train_lbl_list_start)

mk_val_ori_list_start = time.time()
val_ori_list = mk_ori_list(val_list, output_type='list')
print('mk_val_ori_list took', time.time() - mk_val_ori_list_start)

mk_val_lbl_list_start = time.time()
val_lbl_list = mk_lbl_list(val_list, output_type='list')
print('mk_val_lbl_list took', time.time() - mk_val_lbl_list_start)

In [ ]:
print(train_ori_list[0].dtype)
print(train_lbl_list[0].dtype)

# 5. SET Train

In [ ]:
ACTIVATION = 'softmax'
FROM_LOGITS = False
OPTIMIZER = 'RMSprop' # Adam
LR = 0.0005
SMOOTH = 1e-5

BACKBONE = 'efficientnetb4'
ENC_FREEZE = False

BATCH_SIZE = 1
ENC_WEIGHTS = 'imagenet'

## 5.1 train fn()

### 5.1.1 show_prediction


In [ ]:
def show_prediction(model, filename:str, class_list:list) :
    print(f'show_prediction -> {filename}')
    ori_img_path = ori_path + filename.replace(LBL_EXT, ORI_EXT)
    lbl_img_path = lbl_path + filename.replace(ORI_EXT, LBL_EXT)

    raw_ori_src = cv.imread(ori_img_path)
    raw_ori_src = cv.resize(raw_ori_src, (INPUT_SHAPE[:2]), interpolation=cv.INTER_NEAREST)
    raw_ori_src = cv.cvtColor(raw_ori_src, cv.COLOR_BGR2RGB) / 255.0

    out = []
    out.append(raw_ori_src)

    final_ori_src = np.concatenate(out, -1)
    print(f'ori.shape -> {final_ori_src.shape}')

    raw_lbl_src = cv.imread(lbl_img_path)
    raw_lbl_src = cv.cvtColor(raw_lbl_src, cv.COLOR_BGR2RGB)
    raw_lbl_src = cv.resize(raw_lbl_src, (INPUT_SHAPE[:2]), interpolation=cv.INTER_NEAREST)
    onehot_lbl_src = RGB2OneHot(raw_lbl_src, RGB_LABELMAP)

    input_src = tf.expand_dims(final_ori_src, 0)
    pred = model.predict(input_src, batch_size = 1)
    pred_mask = create_mask(pred)

    # preview
    f, axes = plt.subplots(1,3, figsize=(18,8))
    axes[0].imshow(final_ori_src)
    # axes[1].imshow(onehot2RGB(raw_lbl_src[:,:,0], RGB_LABELMAP, bChannel=False))
    axes[1].imshow(raw_lbl_src)
    axes[2].imshow(onehot2RGB(pred_mask[:,:,0], RGB_LABELMAP, bChannel=False))
    plt.show()

    iou_list, class_list = calculate_class_iou(onehot_lbl_src, pred_mask, class_list, preview=False)
    print(iou_list)
    

### 5.1.2 DisplayCallback

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self, model):
        self.model = model
    
    def on_epoch_end(self, epoch, logs = None) :
        loss = np.round(logs['loss'], 3)
        iou = np.round(logs['iou_score'], 3)
        val_loss = np.round(logs['val_loss'], 3)
        val_iou = np.round(logs['val_iou_score'], 3)

        print(f'\nEpoch[{epoch+1}] loss = {loss}, iou ={iou}, val_loss = {val_loss}, val_iou = {val_iou}')

        print('train data ====>')
        for idx in range(1) :
            show_prediction(self.model, train_list[random.randint(0, len(train_list)-1)], TARGET_CLASSES[1:])
        print('val data ====>')
        for idx in range(1) :
            show_prediction(self.model, val_list[random.randint(0, len(val_list)-1)], TARGET_CLASSES[1:])

In [ ]:
model_save_path = './models/'
try :
    os.mkdir(model_save_path)
except :
    print('already exist')

### model cb

In [ ]:
cb_val_loss_cheakpoint = ModelCheckpoint(filepath=model_save_path+f'saved_models_'+'VAL_LOSS'+'_{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, mode='max', save_best_only=True)
cb_val_iou_score_cheakpoint = ModelCheckpoint(filepath=model_save_path+f'saved_models_'+'VAL_IOU_SCORE'+'_{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_iou_socre', verbose=1, mode='max', save_best_only=True)

## custom dice

In [ ]:
class CustomDiceLoss(Loss) :
    def __init__(self, beta=1, class_weights=None, class_indices=None, per_image=False, smooth=1e-5):
        super().__init__(name='dice_loss')
        self.beta = beta
        self.class_weights = class_weights
        self.class_indices = class_indices
        self.per_image = per_image
        self.smooth = smooth

    def __call__(self, gt, pred) :
        return 1 - F.f_score(gt, pred, beta=self.beta, class_weights=self.class_weights, class_indexes=self.class_indices, smooth=self.smooth, per_image=self.per_image, **self.submodules)





### weight change

In [ ]:
'''
    작성은 했지만 어떻게 사용하는지 몰라요 ㅠㅠ
'''

def weight_change(base_model, change_model, num_channles = 3):
    base_model_weight = base_model.get_weights()

    new_weight = np.zeros((3,3,num_channles,32))
    for idx in range(num_channles):
        new_weight[:,:,idx,:] = base_model_weight[idx][:,:,idx,:]

    base_model_weight[0] = new_weight

    change_model.set_weights(base_model_weight)
    return change_model

### create model

In [ ]:
def create_model() :

    if SRC_CHANNELS == 3 :
        base_model = sm.Unet(BACKBONE, input_shape=INPUT_SHAPE, classes=CLASS_CNT, encoder_weights=ENC_WEIGHTS, activation=ACTIVATION, encoder_freeze=ENC_FREEZE)

    else :
        pass

    inp = Input(shape=INPUT_SHAPE[:2]+(SRC_CHANNELS,)) # 3channel color
    print(f'input_shape -> {INPUT_SHAPE[:2]+(SRC_CHANNELS,)}')

    conv2d = Conv2D(SRC_CHANNELS, (1,1))(inp)
    out = base_model(conv2d)

    model = Model(inp, out)

    return model


## set opt

In [ ]:

if OPTIMIZER == 'Adam':
    opt = keras.optimizers.Adam(learning_rate=LR)
if OPTIMIZER == 'RMSprop':
    opt = keras.optimizers.RMSprop(learning_rate=LR)

print(opt)

## model cre

In [ ]:
model = create_model()

## model compile

In [ ]:
model.compile(optimizer=opt,
    # loss = sm.losses.DiceLoss() + sm.losses.CategoricalCELoss(),
    # loss = sm.losses.CategoricalCELoss(class_weights=np.array([7, 16, 14, 7, 7, 14, 14, 14, 7])/100)
    loss = CustomDiceLoss(),
    # loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy', sm.metrics.IOUScore(), sm.metrics.FScore()],
)

In [ ]:
model.summary()

In [ ]:
model.layers[2].layers[-2].output_shape

# cdg

In [ ]:
cdg_train = CustomDataGenerator(train_ori_list, train_lbl_list, image_size=INPUT_SHAPE, shuffle= False, channel_num=3, batch_size=2)
cdg_val = CustomDataGenerator(val_ori_list, val_lbl_list, image_size=INPUT_SHAPE, shuffle= False, channel_num=3, batch_size=1)

# ? . TRAIN

## ?.test

In [ ]:
test_x, test_y = cdg_train.__getitem__(2)

f, axes = plt.subplots(1, 2, figsize=(18,18))
axes[0].imshow(test_x[0].numpy())
axes[1].imshow(onehot2RGB(test_y[0].numpy(), RGB_LABELMAP, bChannel=True))

plt.show()

# print(cv.imwrite('./test.png', test_y[0].numpy()))

# print(test_y[0].numpy()[200][200])

In [ ]:
history_list = []

In [ ]:
print(cdg_train.__getitem__(0)[0].numpy().dtype)
print(cdg_train.__getitem__(0)[1].numpy().dtype)
print(cdg_val.__getitem__(0)[0].numpy().dtype)
print(cdg_val.__getitem__(0)[1].numpy().dtype)

print(cdg_train.__getitem__(0)[0].numpy().shape)
print(cdg_train.__getitem__(0)[1].numpy().shape)
print(cdg_val.__getitem__(0)[0].numpy().shape)
print(cdg_val.__getitem__(0)[1].numpy().shape)

In [ ]:
train_start_time = time.time()
history = model.fit(cdg_train, epochs=200, validation_data=cdg_val,
                    callbacks=[DisplayCallback(model)]
                    )

# 